# Capstone Project - Part 1

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0          conda-forge
    geopy:           

#### Import Data

In [30]:
data = [['1','Centro','45.0688914','7.6823806'],['1','Crocetta','45.0579245','7.6649673'],['2','Santa Rita','45.0456578','7.6455291'],
        ['2','Mirafiori Nord','45.0403513','7.628422'],['2','Mirafiori Sud','45.0158062','7.6361011'],['3','San Paolo','45.0713215','7.6125089'],
        ['3','Cenisia','45.0697067','7.6498785'],['3','Pozzo Strada','45.0749418','7.6291804'],['3','Cit Turin','45.0734308','7.6615327'],
        ['3','Borgata Lesna','45.062501','7.6200512'],['4','San Donato','45.0823608','7.6617867'],['4','Campidoglio','45.0793485','7.6529994'],
        ['4','Parella','45.430657','7.793217'],['5','Borgo Vittoria','45.0973','7.6780633'],['5','Madonna di Campagna','45.1038648','7.6634925'],
        ['5','Lucento','45.098587','7.6480041'],['5','Vallette','45.1043837','7.6322966'],['6','Barriera di Milano','45.0923875','7.6953196'],
        ['6','Regio Parco','45.0930538','7.7149486'],['6','Barca','45.101778','7.7278795'],['6','Bertolla','45.0992234','7.7478559'],
        ['6','Falchera','45.1283575','7.7093621'],['6','Rebaudengo','45.1042567','7.6993038'],['6','Villaretto','45.1313067','7.6850598'],
        ['7','Aurora','45.0825745','7.6860091'],['7','Vanchiglia','45.0704376','7.6984692'],['7','Sassi','45.079322','7.7323063'],
        ['7','Madonna del Pilone','45.068287','7.7201896'],['8','San Salvario','45.0549503','7.6801517'],['8','Cavoretto','44.8662114','7.2891873'],
        ['8','Borgo Po','45.0606675','7.6991276'],['8','Nizza Millefonti','45.030171','7.6678494'],['8','Lingotto','45.0265963','7.6568231'],
        ['8','Filadelfia','45.0370503','7.6570621']]

df = pd.DataFrame(data,columns=['Borough','Neighborhood','Latitude','Longitude'])
df.head()

,Borough,Neighborhood,Latitude,Longitude
0,1,Centro,45.0688914,7.6823806
1,1,Crocetta,45.0579245,7.6649673
2,2,Santa Rita,45.0456578,7.6455291
3,2,Mirafiori Nord,45.0403513,7.628422
4,2,Mirafiori Sud,45.0158062,7.6361011


# Capstone Project - Part 2

 #### Use geopy library to get the latitude and longitude values of Turin.

In [31]:
address = 'Turin, IT'

geolocator = Nominatim(user_agent="tu_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Turin are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Turin are 45.0677551, 7.6824892.


#### Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

In [32]:
CLIENT_ID = 'QP22UJUREYK2TP2MNV3EO40DJZYE3IWIKBVUOZTAGAXLEDR0' # your Foursquare ID
CLIENT_SECRET = 'HJJ5PIOEX4TJVB1V4O4Z41YSTDJYUUYCNF1B4B0RQB4MWTV3' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: QP22UJUREYK2TP2MNV3EO40DJZYE3IWIKBVUOZTAGAXLEDR0
CLIENT_SECRET:HJJ5PIOEX4TJVB1V4O4Z41YSTDJYUUYCNF1B4B0RQB4MWTV3


#### Let's get the top 200 venues in each Neighbourhood of Turin within a radius of 500 meters.

In [33]:
LIMIT=200

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [8]:
turin_venues = getNearbyVenues(names=df['Neighbourhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Centro
Crocetta
Santa Rita
Mirafiori Nord
Mirafiori Sud
San Paolo
Cenisia
Pozzo Strada
Cit Turin
Borgata Lesna
San Donato
Campidoglio
Parella
Borgo Vittoria
Madonna di Campagna
Lucento
Vallette
Barriera di Milano
Regio Parco
Barca
Bertolla
Falchera
Rebaudengo
Villaretto
Aurora
Vanchiglia
Sassi
Madonna del Pilone
San Salvario
Cavoretto
Borgo Po
Nizza Millefonti
Lingotto
Filadelfia


In [34]:
print(turin_venues.shape)
turin_venues.head(15)

(590, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Centro,45.0688914,7.6823806,Piazza San Carlo,45.067804,7.682546,Plaza
1,Centro,45.0688914,7.6823806,Caffé Stratta,45.067552,7.682555,Café
2,Centro,45.0688914,7.6823806,Farmacia del Cambio,45.069554,7.684915,Dessert Shop
3,Centro,45.0688914,7.6823806,Galleria San Federico,45.068984,7.682887,Historic Site
4,Centro,45.0688914,7.6823806,Museo Egizio,45.068344,7.684363,History Museum
5,Centro,45.0688914,7.6823806,Piazza Carignano,45.069270,7.685010,Plaza
6,Centro,45.0688914,7.6823806,Caffè San Carlo,45.068402,7.682523,Café
7,Centro,45.0688914,7.6823806,Apple via Roma,45.069211,7.683386,Electronics Store
8,Centro,45.0688914,7.6823806,Gelateria La Romana,45.068704,7.682292,Ice Cream Shop
9,Centro,45.0688914,7.6823806,Cinema Lux,45.068929,7.682846,Multiplex


#### Let's count the venues actually found in each Neighbourhood of Turin

In [35]:
turin_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Aurora,20,20,20,20,20,20
Barca,4,4,4,4,4,4
Barriera di Milano,7,7,7,7,7,7
Bertolla,4,4,4,4,4,4
Borgata Lesna,5,5,5,5,5,5
Borgo Po,40,40,40,40,40,40
Borgo Vittoria,5,5,5,5,5,5
Campidoglio,22,22,22,22,22,22
Cenisia,20,20,20,20,20,20


In [36]:
print('There are {} uniques categories.'.format(len(turin_venues['Venue Category'].unique())))

There are 144 uniques categories.


#### Venue Analysis

In [37]:
# one hot encoding
turin_onehot = pd.get_dummies(turin_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
turin_onehot['Neighborhood'] = turin_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [turin_onehot.columns[-1]] + list(turin_onehot.columns[:-1])
turin_onehot = turin_onehot[fixed_columns]

turin_onehot.head()

,Neighborhood,Abruzzo Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Auditorium,Auto Workshop,Bakery,Bank,Bar,Baseball Stadium,Bike Rental / Bike Share,Bistro,Bookstore,Boutique,Bowling Green,Breakfast Spot,Brewery,Burger Joint,Bus Station,Bus Stop,Café,Campanian Restaurant,Campground,Castle,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Community Center,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cultural Center,Cupcake Shop,Deli / Bodega,Dessert Shop,Diner,Discount Store,Eastern European Restaurant,Electronics Store,Emilia Restaurant,Ethiopian Restaurant,Event Space,Fast Food Restaurant,Flea Market,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gastropub,German Restaurant,Gift Shop,Gourmet Shop,Gym,Gym / Fitness Center,Hawaiian Restaurant,Health Food Store,Herbs & Spices Store,Historic Site,History Museum,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Kids Store,Latin American Restaurant,Light Rail Station,Lounge,Market,Massage Studio,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Miscellaneous Shop,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Venue,Nightclub,Nightlife Spot,Noodle House,Office,Opera House,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Peruvian Restaurant,Pet Store,Photography Lab,Piadineria,Piedmontese Restaurant,Pizza Place,Playground,Plaza,Pool,Pub,Public Art,Racetrack,Recreation Center,Restaurant,Rock Club,Roman Restaurant,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Sicilian Restaurant,Skate Park,Smoke Shop,Soccer Field,Soccer Stadium,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Theater,Toy / Game Store,Train Station,Trattoria/Osteria,Vegetarian / Vegan Restaurant,Waterfront,Wine Bar,Wine Shop,Winery
0,Centro,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Centro,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Centro,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Centro,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Centro,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [38]:
turin_onehot.shape

(590, 145)

#### Let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [39]:
turin_grouped = turin_onehot.groupby('Neighborhood').mean().reset_index()
turin_grouped

,Neighborhood,Abruzzo Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Auditorium,Auto Workshop,Bakery,Bank,Bar,Baseball Stadium,Bike Rental / Bike Share,Bistro,Bookstore,Boutique,Bowling Green,Breakfast Spot,Brewery,Burger Joint,Bus Station,Bus Stop,Café,Campanian Restaurant,Campground,Castle,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Community Center,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cultural Center,Cupcake Shop,Deli / Bodega,Dessert Shop,Diner,Discount Store,Eastern European Restaurant,Electronics Store,Emilia Restaurant,Ethiopian Restaurant,Event Space,Fast Food Restaurant,Flea Market,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gastropub,German Restaurant,Gift Shop,Gourmet Shop,Gym,Gym / Fitness Center,Hawaiian Restaurant,Health Food Store,Herbs & Spices Store,Historic Site,History Museum,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Kids Store,Latin American Restaurant,Light Rail Station,Lounge,Market,Massage Studio,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Miscellaneous Shop,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Venue,Nightclub,Nightlife Spot,Noodle House,Office,Opera House,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Peruvian Restaurant,Pet Store,Photography Lab,Piadineria,Piedmontese Restaurant,Pizza Place,Playground,Plaza,Pool,Pub,Public Art,Racetrack,Recreation Center,Restaurant,Rock Club,Roman Restaurant,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Sicilian Restaurant,Skate Park,Smoke Shop,Soccer Field,Soccer Stadium,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Theater,Toy / Game Store,Train Station,Trattoria/Osteria,Vegetarian / Vegan Restaurant,Waterfront,Wine Bar,Wine Shop,Winery
0,Aurora,0.0,0.000000,0.00,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.150000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.050000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.050000,0.00,0.000000,0.0,0.050000,0.000000,0.000000,0.050000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.050000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.05,0.000000,0.00,0.000000,0.000000,0.000,0.000000,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.05,0.000000,0.0,0.100000,0.050000,0.000000,0.00,0.000000,0.050000,0.000000,0.00,0.000000,0.000000,0.050000,0.000000,0.00,0.000000,0.000000,0.000000,0.050000,0.000000,0.000000,0.050000,0.000000,0.000,0.000000,0.000000,0.000,0.000000,0.000000,0.0
1,Barca,0.0,0.000000,0.00,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000,0.000000,0.000,0

#### Let's print each neighborhood along with the top 10 most common venues, then let's put that into a *pandas* dataframe

In [40]:
num_top_venues = 10

for hood in turin_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = turin_grouped[turin_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Aurora----
                 venue  freq
0   Chinese Restaurant  0.15
1                Diner  0.10
2           Restaurant  0.10
3        Historic Site  0.05
4       Scenic Lookout  0.05
5            Rock Club  0.05
6           Skate Park  0.05
7  Moroccan Restaurant  0.05
8           Public Art  0.05
9          Flea Market  0.05


----Barca----
                 venue  freq
0                 Café  0.50
1  Japanese Restaurant  0.25
2         Dessert Shop  0.25
3           Piadineria  0.00
4      Photography Lab  0.00
5            Pet Store  0.00
6  Peruvian Restaurant  0.00
7     Pedestrian Plaza  0.00
8          Pastry Shop  0.00
9               Museum  0.00


----Barriera di Milano----
                 venue  freq
0          Pizza Place  0.14
1     Kebab Restaurant  0.14
2           Smoke Shop  0.14
3            Pet Store  0.14
4               Market  0.14
5     Community Center  0.14
6  Fried Chicken Joint  0.14
7          Opera House  0.00
8            Nightclub  0.00
9       Nigh

In [41]:
# function to sort the venue in descending order

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [42]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = turin_grouped['Neighborhood']

for ind in np.arange(turin_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(turin_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Aurora,Chinese Restaurant,Restaurant,Diner,Scenic Lookout,Supermarket,Moroccan Restaurant,Public Art,Lounge,Skate Park,Flea Market
1,Barca,Café,Japanese Restaurant,Dessert Shop,Winery,Event Space,Food Stand,Food Court,Food & Drink Shop,Flea Market,Fast Food Restaurant
2,Barriera di Milano,Market,Pet Store,Pizza Place,Fried Chicken Joint,Community Center,Kebab Restaurant,Smoke Shop,Winery,Flea Market,Fast Food Restaurant
3,Bertolla,Pizza Place,Pub,Gym / Fitness Center,Massage Studio,Winery,Emilia Restaurant,Food & Drink Shop,Flea Market,Fast Food Restaurant,Event Space
4,Borgata Lesna,Ice Cream Shop,Japanese Restaurant,Trattoria/Osteria,Kebab Restaurant,Restaurant,Ethiopian Restaurant,Food Court,Food & Drink Shop,Flea Market,Fast Food Restaurant
5,Borgo Po,Cocktail Bar,Italian Restaurant,Café,Piedmontese Restaurant,Nightclub,Japanese Restaurant,Event Space,Sandwich Place,Pub,Plaza
6,Borgo Vittoria,Winery,Pizza Place,Café,Kids Store,Pub,Emilia Restaurant,Food & Drink Shop,Flea Market,Fast Food Restaurant,Event Space
7,Campidoglio,Café,Hotel,Plaza,Irish Pub,Jewelry Store,Diner,Clothing Store,Pizza Place,Restaurant,Rock Club
8,Cenisia,Café,Ice Cream Shop,Bike Rental / Bike Share,Kebab Restaurant,French Restaurant,Cocktail Bar,Plaza,Market,Coffee Shop,Pizza Place
9,Centro,Café,Italian Restaurant,Ice Cream Shop,Plaza,Historic Site,Piedmontese Restaurant,Hotel,History Museum,Sandwich Place,Bookstore


### Cluster Neighbourhoods

In [43]:
# set number of clusters
kclusters = 5

turin_grouped_clustering = turin_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(turin_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [96]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster__ Label', kmeans.labels_)

turin_merged = df

# merge turin_grouped with df to add latitude/longitude for each neighborhood
turin_merged = turin_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

turin_merged.head(10)

,Borough,Neighborhood,Latitude,Longitude,Cluster__ Label,Cluster_ Label,Cluster_Label,Clusters_Label,Cluster_Labels,Clusters Label,Clusters Labels,Cluster Labels,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,Centro,45.0688914,7.6823806,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Café,Italian Restaurant,Ice Cream Shop,Plaza,Historic Site,Piedmontese Restaurant,Hotel,History Museum,Sandwich Place,Bookstore
1,1,Crocetta,45.0579245,7.6649673,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Café,Ice Cream Shop,Pizza Place,Coffee Shop,Sushi Restaurant,Monument / Landmark,Pub,Market,Bar,Supermarket
2,2,Santa Rita,45.0456578,7.6455291,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Pizza Place,Café,Ice Cream Shop,Supermarket,Gourmet Shop,Plaza,Clothing Store,Fast Food Restaurant,Fried Chicken Joint,Gastropub
3,2,Mirafiori Nord,45.0403513,7.628422,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Pizza Place,Plaza,Gym,Event Space,Bakery,Health Food Store,Seafood Restaurant,Ice Cream Shop,Italian Restaurant,Hotel
4,2,Mirafiori Sud,45.0158062,7.6361011,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Food Stand,IT Services,Construction & Landscaping,Food & Drink Shop,Recreation Center,Emilia Restaurant,Food Court,Flea Market,Fast Food Restaurant,Event Space
5,3,San Paolo,45.0713215,7.6125089,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Pizza Place,Café,Discount Store,Metro Station,Pub,Bakery,Bus Station,Winery,Fast Food Restaurant,Food Court
6,3,Cenisia,45.0697067,7.6498785,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Café,Ice Cream Shop,Bike Rental / Bike Share,Kebab Restaurant,French Restaurant,Cocktail Bar,Plaza,Market,Coffee Shop,Pizza Place
7,3,Pozzo Strada,45.0749418,7.6291804,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Supermarket,Café,Market,Indian Restaurant,Asian Restaurant,Chinese Restaurant,Metro Station,Pizza Place,Boutique,Clothing Store
8,3,Cit Turin,45.0734308,7.6615327,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Café,Piedmontese Restaurant,Bus Station,Sushi Restaurant,Pizza Place,Plaza,Chinese Restaurant,Italian Restaurant,Bus Stop,Market
9,3,Borgata Lesna,45.062501,7.6200512,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Ice Cream Shop,Japanese Restaurant,Trattoria/Osteria,Kebab Restaurant,Restaurant,Ethiopian Restaurant,Food Court,Food & Drink Shop,Flea Market,Fast Food Restaurant


#### Examine Clusters

In [105]:
#first cluster

turin_merged.loc[turin_merged['Cluster__ Label'] == 0, turin_merged.columns[[1] + list(range(5, turin_merged.shape[1]))]]

,Neighborhood,Cluster_ Label,Cluster_Label,Clusters_Label,Cluster_Labels,Clusters Label,Clusters Labels,Cluster Labels,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Villaretto,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Restaurant,Cultural Center,Food Stand,Food Court,Food & Drink Shop,Flea Market,Fast Food Restaurant,Event Space,Ethiopian Restaurant,Winery


In [106]:
#second cluster

turin_merged.loc[turin_merged['Cluster__ Label'] == 1, turin_merged.columns[[1] + list(range(5, turin_merged.shape[1]))]]

,Neighborhood,Cluster_ Label,Cluster_Label,Clusters_Label,Cluster_Labels,Clusters Label,Clusters Labels,Cluster Labels,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Centro,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Café,Italian Restaurant,Ice Cream Shop,Plaza,Historic Site,Piedmontese Restaurant,Hotel,History Museum,Sandwich Place,Bookstore
1,Crocetta,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Café,Ice Cream Shop,Pizza Place,Coffee Shop,Sushi Restaurant,Monument / Landmark,Pub,Market,Bar,Supermarket
2,Santa Rita,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Pizza Place,Café,Ice Cream Shop,Supermarket,Gourmet Shop,Plaza,Clothing Store,Fast Food Restaurant,Fried Chicken Joint,Gastropub
3,Mirafiori Nord,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Pizza Place,Plaza,Gym,Event Space,Bakery,Health Food Store,Seafood Restaurant,Ice Cream Shop,Italian Restaurant,Hotel
4,Mirafiori Sud,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Food Stand,IT Services,Construction & Landscaping,Food & Drink Shop,Recreation Center,Emilia Restaurant,Food Court,Flea Market,Fast Food Restaurant,Event Space
5,San Paolo,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Pizza Place,Café,Discount Store,Metro Station,Pub,Bakery,Bus Station,Winery,Fast Food Restaurant,Food Court
6,Cenisia,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Café,Ice Cream Shop,Bike Rental / Bike Share,Kebab Restaurant,French Restaurant,Cocktail Bar,Plaza,Market,Coffee Shop,Pizza Place
7,Pozzo Strada,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Supermarket,Café,Market,Indian Restaurant,Asian Restaurant,Chinese Restaurant,Metro Station,Pizza Place,Boutique,Clothing Store
8,Cit Turin,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Café,Piedmontese Restaurant,Bus Station,Sushi Restaurant,Pizza Place,Plaza,Chinese Restaurant,Italian Restaurant,Bus Stop,Market
9,Borgata Lesna,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Ice Cream Shop,Japanese Restaurant,Trattoria/Osteria,Kebab Restaurant,Restaurant,Ethiopian Restaurant,Food Court,Food & Drink Shop,Flea Market,Fast Food Restaurant


In [107]:
#third cluster

turin_merged.loc[turin_merged['Cluster__ Label'] == 2, turin_merged.columns[[1] + list(range(5, turin_merged.shape[1]))]]

,Neighborhood,Cluster_ Label,Cluster_Label,Clusters_Label,Cluster_Labels,Clusters Label,Clusters Labels,Cluster Labels,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,Lucento,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,Italian Restaurant,Soccer Stadium,Winery,Ethiopian Restaurant,Food Stand,Food Court,Food & Drink Shop,Flea Market,Fast Food Restaurant,Event Space
21,Falchera,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,Italian Restaurant,Bus Station,Cupcake Shop,Food Truck,Food Stand,Food Court,Food & Drink Shop,Flea Market,Fast Food Restaurant,Event Space


In [108]:
#fourth cluster

turin_merged.loc[turin_merged['Cluster__ Label'] == 3, turin_merged.columns[[1] + list(range(5, turin_merged.shape[1]))]]

,Neighborhood,Cluster_ Label,Cluster_Label,Clusters_Label,Cluster_Labels,Clusters Label,Clusters Labels,Cluster Labels,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Parella,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,Hotel,Italian Restaurant,Ethiopian Restaurant,Food Stand,Food Court,Food & Drink Shop,Flea Market,Fast Food Restaurant,Event Space,Emilia Restaurant


In [109]:
#fifth cluster

turin_merged.loc[turin_merged['Cluster__ Label'] == 4, turin_merged.columns[[1] + list(range(5, turin_merged.shape[1]))]]

,Neighborhood,Cluster_ Label,Cluster_Label,Clusters_Label,Cluster_Labels,Clusters Label,Clusters Labels,Cluster Labels,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,Vallette,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,Lounge,French Restaurant,Food Stand,Food Court,Food & Drink Shop,Flea Market,Fast Food Restaurant,Event Space,Ethiopian Restaurant,Winery
